In [1]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import json

In [2]:
FILE_DIR_PATH = './data/'
INPUT_TRAIN_DATA_FILE_NAME = 'input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'label.npy'
DATA_CONFIGS_FILE_NAME = 'data_configs.json'

In [4]:
TEST_SPLIT = 0.1
RNG_SEED = 13371447

In [21]:
input_data = np.load(open(FILE_DIR_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
label_data = np.load(open(FILE_DIR_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'rb'))
prepro_configs = None

with open(FILE_DIR_PATH + DATA_CONFIGS_FILE_NAME, 'r') as f:
    prepro_configs = json.load(f)
    print(prepro_configs.keys())

dict_keys(['vocab', 'vocab_size'])


In [5]:
input_train, input_test, label_train, label_test = train_test_split(input_data, label_data, test_size=TEST_SPLIT, random_state=RNG_SEED)

In [15]:
BATCH_SIZE = 16
NUM_EPOCHS = 10

def mapping_fn(X, Y):
    input, label = {'text': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_test, label_test))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [22]:
model_dir = './models/rnn_classifier'
vocab_size = prepro_configs['vocab_size']
embedding_size = 50

# pre-made estimator로 평가를 할 때, 일정한 함수를 사용하게 세팅합니다.
head = tf.contrib.estimator.binary_classification_head()

def rnn_model_fn(features, labels, mode, params):
    #embedding layer를 선언합니다.
    input_layer = tf.contrib.layers.embed_sequence(
                    features['text'],
                    vocab_size,
                    embedding_size,
                    initializer=params['embedding_initializer']
                    )
    # 현재 모델이 학습모드인지 여부를 확인하는 변수입니다.
    training = (mode == tf.estimator.ModeKeys.TRAIN)
    # embedding layer에 대한 output에 대해 dropout을 취합니다.
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.2,
                                   training=training)
    
    # LSTM Cell Network를 선언합니다. 객체 생성자 파라메터에는 해당 hidden dimension을 정의해둡니다.
    # Multi Layer의 경우엔 아래와 같이 List 형태로 저장을 해둡니다.
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [100, 100]]

    # Multi RNN Cell이 복수의 LSTM Cell를 하나의 Cell Network로 선언해줍니다.
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

    # 네트워크와 embedding vector와 연산하기 위해 dynamic_rnn 함수를 선언합니다.
    outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                       inputs=dropout_emb,
                                       dtype=tf.float32)
    
    # rnn network를 거쳐 나온 마지막 step의 hidden vector를 활용하여 감정을 분석하고자 합니다.
    # rnn network에서 나온 벡터들 중 마지막 step에 있는 vector만을 가지고 fully connected를 거치도록 합니다.
    hidden = tf.layers.dense(inputs=outputs[:,-1,:], units=100, activation=tf.nn.relu)  
    dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
    # 최종적으로 1 dim의 output만 할 수 있도록 dense layer를 활용하여 차원 변환을 해줍니다.
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    # evaluation을 하는 상황이 아닌 경우 label 벡터에 대한 값의 dimension을 맞춰줍니다.
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
    
    # Optimizer를 지정해줍니다. 하이퍼파라메터로 다른 Optimizer를 지정해줄 수 있습니다.
    optimizer = tf.train.AdamOptimizer() 
    
    def _train_op_fn(loss):
        return optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())

    # pre-made estimator를 활용하여 train 및 evaluation을 진행합니다.
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)

In [25]:
# estimator에는 params라는 파라메터를 전달할 수 있도록 정의되어있습니다. 
# 여러분이 해당 모델에 설정하고자 하는 하이퍼파라메터에 대해 각각 지정할 내용을 dictionary 형태로 
# 선언을 하면 모델 함수 내부에서 활용이 가능합니다.

# 여기선 embedding layer에 파라메터에 대해 initialize를 설정해줍니다.
# 이 경우는 -1~1 사이의 uniform 분포로 초기화 합니다.
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}

# tf.estimator 객체를 선언합니다. 모델과 저장할 모델 저장 위치를 선언해줍니다.
rnn_classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,
                                        model_dir=model_dir, 
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/rnn_classifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11bd02358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
rnn_classifier.train(train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./models/rnn_classifier/model.ckpt.
INFO:tensorflow:loss = 11.094627, step = 1


KeyboardInterrupt: 